In [2]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, GRPO_CONNECTIONS_CONFIG, SMOL_LM_135M

cfg = GRPO_CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-03-25 21:28:39.302 | INFO     | trl_wrapper.trainer_wrapper:__init__:266 - Setting padding side to: left
2025-03-25 21:28:39.312 | INFO     | trl_wrapper.trainer_wrapper:init_model:277 - Using device: mps
2025-03-25 21:28:39.313 | INFO     | trl_wrapper.trainer_wrapper:init_model:291 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [9]:
import pandas as pd

pd.read_json("../dataset_files/connections_prompts.jsonl", lines=True)

,words,solution
0,"[nets, return, heat, jazz, mom, shift, kayak, ...","{'groups': [{'words': ['bucks', 'heat', 'jazz'..."
1,"[league, loafer, queue, are, pump, foot, why, ...","{'groups': [{'words': ['foot', 'league', 'mile..."
2,"[amigo, wolf, cheek, pom, stooge, lab, king, p...","{'groups': [{'words': ['chow', 'gobble', 'scar..."
3,"[puma, sweep, chicago, bat, iron, adidas, vacu...","{'groups': [{'words': ['cabaret', 'carousel', ..."
4,"[glum, prime, low, hulu, green, plum, blue, do...","{'groups': [{'words': ['ketchup', 'mayo', 'rel..."
...,...,...
348,"[wee, back, champion, poster, first, here, pre...","{'groups': [{'words': ['first', 'initial', 'ma..."
349,"[griddle, fringe, tease, geez, seize, pan, ket...","{'groups': [{'words': ['frill', 'fringe', 'ruf..."
350,"[quarter, down, flat, choice, natural, heel, t...","{'groups': [{'words': ['down', 'heel', 'shake'..."
351,"[evil, scented, receptive, flexible, vile, wax...","{'groups': [{'words': ['evil', 'live', 'veil',..."


In [24]:
wrapper.init_data_module()

2025-03-25 21:34:25.351 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:317 - Using chat template override: smollmv2
2025-03-25 21:34:25.352 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 16892.45 examples/s]


In [25]:
wrapper.data_module.train_dataset[0]

{'words': ['risk',
  'bond',
  'trouble',
  'sorry',
  'smart',
  'tongue',
  'soul',
  'cement',
  'duckie',
  'band',
  'hunt',
  'sole',
  'clue',
  'lace',
  'heel',
  'ryan'],
 'solution': {'groups': [{'reason': 'board games',
    'words': ['clue', 'risk', 'sorry', 'trouble']},
   {'reason': 'shoe parts', 'words': ['heel', 'lace', 'sole', 'tongue']},
   {'reason': 'fictional spies', 'words': ['bond', 'hunt', 'ryan', 'smart']},
   {'reason': 'rubber ___', 'words': ['band', 'cement', 'duckie', 'soul']}]},
 'prompt': [{'content': '\nYou are an expert puzzle solving model.\nFind groups of words that are related to each other, and return the answer in the following format:\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n# Example\n\nUser: apple, orange, banana, pear, corolla, charger,\nAssistant: <reasoning>\nThe first group are all fruits.\nThe second group are all cars.\n</reasoning>\n<answer>\n

In [27]:
wrapper.init_trainer()

2025-03-25 21:35:05.279 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:354 - Saving output to: ./runs/03-25-21-35-952469-smollm2-135m-instruct-
2025-03-25 21:35:05.280 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:377 - Initializing trainer, run_name: 03-25-21-35-952469-smollm2-135m-instruct-, wandb project: qwen-connections-grpo
2025-03-25 21:35:05.280 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:462 - Using vllm: False


In [28]:
from model.reasoning import SYSTEM_PROMPT

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_attention_mask=True
)
tokenized_chat = tokenized_chat.to("cuda")
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
wrapper.train()

2025-03-25 21:35:29.499 | INFO     | trl_wrapper.trainer_wrapper:train:626 - Starting training.


{'words': ['tombstone', 'cherry', '20,000', 'bat', 'firefly', 'bell', '24', '451', 'cobweb', 'bar', '7', 'bones', '22', '2001', 'weeds', 'pumpkin'], 'solution': {'groups': [{'reason': 'tv shows', 'words': ['24', 'bones', 'firefly', 'weeds']}, {'reason': 'halloween decorations', 'words': ['bat', 'cobweb', 'pumpkin', 'tombstone']}, {'reason': 'slot machine symbols', 'words': ['7', 'bar', 'bell', 'cherry']}, {'reason': 'numbers in book titles', 'words': ['22', '451', '2001', '20,000']}]}, 'prompt': [{'content': '\nYou are an expert puzzle solving model.\nFind groups of words that are related to each other, and return the answer in the following format:\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n# Example\n\nUser: apple, orange, banana, pear, corolla, charger,\nAssistant: <reasoning>\nThe first group are all fruits.\nThe second group are all cars.\n</reasoning>\n<answer>\n<group>apple, orange, ba